## Semantic Router Dynamic Routing

Allows you to dynamically set variables to use in functions

In [1]:
import os

from dotenv import load_dotenv
from datetime import datetime
from zoneinfo import ZoneInfo
from semantic_router import Route
from semantic_router.layer import RouteLayer
from semantic_router.encoders import OpenAIEncoder
from semantic_router.utils.function_call import get_schema
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory

In [2]:
load_dotenv()

True

### Set up functions: Must be in sphinx

In [3]:
def get_time(timezone: str) -> str:
    """Finds the current time in a specific timezone.

    :param timezone: The timezone to find the current time in, should
        be a valid timezone from the IANA Time Zone Database like
        "America/New_York" or "Europe/London". Do NOT put the place
        name itself like "rome", or "new york", you must provide
        the IANA format.
    :type timezone: str
    :return: The current time in the specified timezone."""
    now = datetime.now(ZoneInfo(timezone))
    return now.strftime("%H:%M")

In [4]:
get_time("America/New_York")

'14:30'

In [5]:
# get the function schema
schema = get_schema(get_time)
schema

{'name': 'get_time',
 'description': 'Finds the current time in a specific timezone.\n\n:param timezone: The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London". Do NOT put the place\n    name itself like "rome", or "new york", you must provide\n    the IANA format.\n:type timezone: str\n:return: The current time in the specified timezone.',
 'signature': '(timezone: str) -> str',
 'output': "<class 'str'>"}

In [6]:
time_route = Route(
    name="get_time",
    utterances=[
        "what is the time in new york city?",
        "what is the time in london?",
        "I live in Rome, what time is it?",
    ],
    function_schema=schema,
)

routes = [time_route]

### Router Example

In [7]:
encoder = OpenAIEncoder()
layer = RouteLayer(encoder=encoder, routes=routes)

2024-01-19 12:30:07 INFO semantic_router.utils.logger Initializing RouteLayer


In [8]:
#layer.add(route=time_route)

In [9]:
print(layer("what is the time in new york city?").name)

2024-01-19 12:30:07 INFO semantic_router.utils.logger Extracting function input...
2024-01-19 12:30:08 ERROR semantic_router.utils.logger LLM error: Error code: 401 - {'error': {'message': 'Missing Authentication header or invalid API key', 'code': 401}}


Exception: LLM error: Error code: 401 - {'error': {'message': 'Missing Authentication header or invalid API key', 'code': 401}}

Our dynamic route provides both the route itself and the input parameters required to use the route.

In [ ]:
out = layer("what is the time in new york city?")
get_time(**out.function_call)